## DIA 015: Implementacion de Data Augmentation para Mejorar el Modelo

El Data Augmentation (Aumento de Datos) es una técnica que genera nuevas muestras a partir de las existentes aplicando transformaciones aleatorias. Esto ayuda a mejorar la generalización del modelo y a prevenir el overfitting (sobreajuste). En este día, implementaremos Data Augmentation en el pipeline de datos de entrenamiento y reeentrenaremos el modelo para observar mejoras en su rendimiento.

In [14]:
# 1. Importación de Librerías
# ------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:
# 2. Verificación de la Disponibilidad de GPU
# ------------------------------------

# Verificar si TensorFlow detecta una GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU(s) disponible(s): {[gpu.name for gpu in gpus]}")
else:
    print("No hay GPU disponible. El entrenamiento se realizará en CPU, lo cual puede ser más lento.")


In [ ]:
# 3. Carga y Preprocesamiento de Datos
# ------------------------------------

# a. Cargar el dataset MNIST
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# b. Normalizar los valores de píxeles de 0-255 a 0-1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# c. Añadir la dimensión de los canales (1 para escala de grises)
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

# d. Verificar la forma de los datos
print("Forma de X_train:", X_train.shape)  # (60000, 28, 28, 1)
print("Forma de X_test:", X_test.shape)    # (10000, 28, 28, 1)


In [ ]:
# 4. Visualización de Datos (Opcional)
# ------------------------------------

# Función para visualizar imágenes
def mostrar_imagen(matriz, etiqueta, indice):
    plt.figure(figsize=(2,2))
    plt.imshow(matriz[indice].reshape(28, 28), cmap='gray')
    plt.title(f"Dígito: {etiqueta}")
    plt.axis('off')
    plt.show()

# Mostrar la primera imagen del conjunto de entrenamiento
mostrar_imagen(X_train, y_train, 0)


In [18]:
# 5. Definir la Función de Preprocesamiento y Data Augmentation
# ------------------------------------

# a. Definir las capas de Data Augmentation utilizando Keras
data_augmentation_layer = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),  # Voltear horizontal y verticalmente
    layers.RandomRotation(0.15),                    # Rotar aleatoriamente hasta 15%
    layers.RandomZoom(0.1),                         # Aplicar zoom aleatorio hasta un 10%
    layers.RandomContrast(0.1),                     # Ajustar el contraste aleatoriamente
])

# b. Función para aplicar preprocesamiento
def preprocesar_imagen(x, y):
    """
    Redimensiona las imágenes a 224x224, convierte de escala de grises a RGB y normaliza.
    
    Args:
        x (tf.Tensor): Imagen de entrada con forma (28, 28, 1).
        y (tf.Tensor): Etiqueta correspondiente.
        
    Returns:
        tuple: Imagen preprocesada y etiqueta.
    """
    # Redimensionar la imagen a 224x224
    x = tf.image.resize(x, [224, 224])
    
    # Convertir de escala de grises a RGB (duplicar el canal)
    x = tf.image.grayscale_to_rgb(x)
    
    # Normalizar los valores de píxeles a [0, 1]
    x = tf.cast(x, tf.float32) / 255.0
    
    return x, y

# c. Función para aplicar Data Augmentation
def aplicar_data_augmentation(x, y):
    """
    Aplica técnicas de Data Augmentation a las imágenes de entrenamiento.
    
    Args:
        x (tf.Tensor): Imagen preprocesada con forma (224, 224, 3).
        y (tf.Tensor): Etiqueta correspondiente.
        
    Returns:
        tuple: Imagen aumentada y etiqueta.
    """
    x = data_augmentation_layer(x)
    return x, y


In [19]:
# 6. Crear los Objetos tf.data.Dataset con Data Augmentation
# ------------------------------------

# a. Crear el dataset de entrenamiento
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(preprocesar_imagen, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(aplicar_data_augmentation, num_parallel_calls=tf.data.AUTOTUNE)  # Aplicar Data Augmentation solo en entrenamiento
train_ds = train_ds.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.AUTOTUNE)  # Reducir batch_size a 32 para CPU

# b. Crear el dataset de validación (sin Data Augmentation)
val_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_ds = val_ds.map(preprocesar_imagen, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(tf.data.AUTOTUNE)  # Reducir batch_size a 32 para CPU


In [ ]:
# 7. Construcción del Modelo con Transfer Learning
# ------------------------------------

# a. Cargar el modelo VGG16 sin las capas superiores y con pesos preentrenados en ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# b. Congelar las capas del modelo base para que no se actualicen durante el entrenamiento inicial
base_model.trainable = False

# c. Construir el modelo completo con Transfer Learning (Sin data_augmentation_layer)
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 10 clases para MNIST
])

# d. Resumen del modelo
model.summary()


In [21]:
# 8. Compilación del Modelo
# ------------------------------------

# Compilar el modelo definiendo el optimizador, la función de pérdida y las métricas
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [22]:
# 9. Definir Early Stopping y ModelCheckpoint
# ------------------------------------

# Definir Early Stopping para detener el entrenamiento si la pérdida de validación no mejora
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Definir ModelCheckpoint para guardar el mejor modelo basado en la pérdida de validación
checkpoint = ModelCheckpoint('mejor_modelo.h5', monitor='val_loss', save_best_only=True, verbose=1)


In [ ]:
# 10. Entrenamiento del Modelo con Data Augmentation
# ------------------------------------

# Entrenar el modelo usando los datasets de entrenamiento y validación
history = model.fit(
    train_ds,
    epochs=20,
    validation_data=val_ds,
    callbacks=[early_stop, checkpoint]
)


In [ ]:
# 11. Evaluación del Modelo
# ------------------------------------

# Evaluar el rendimiento en el conjunto de prueba
test_loss, test_acc = model.evaluate(val_ds, verbose=2)
print(f'\nPrecisión en el conjunto de prueba: {test_acc:.4f}')


In [ ]:
# 12. Visualización del Rendimiento del Modelo
# ------------------------------------

# Graficar precisión y pérdida durante el entrenamiento
plt.figure(figsize=(12, 4))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión durante el Entrenamiento con Data Augmentation')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

# Pérdida
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida durante el Entrenamiento con Data Augmentation')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

In [ ]:
# 13. Guardado y Carga del Mejor Modelo
# ------------------------------------

# b. Cargar el mejor modelo guardado
mejor_modelo = tf.keras.models.load_model('mejor_modelo.h5')
print("Mejor modelo cargado exitosamente.")


In [ ]:
# 14. Hacer Predicciones con el Mejor Modelo Cargado
# ------------------------------------

# a. Hacer predicciones sobre el conjunto de prueba
predicciones = mejor_modelo.predict(val_ds)
clases_predichas = np.argmax(predicciones, axis=1)

# b. Mostrar predicción para la primera imagen del conjunto de prueba
indice = 0
print(f"Etiqueta real: {y_test[indice]}")
print(f"Predicción: {clases_predichas[indice]}")
print(f"Probabilidades: {predicciones[indice]}")

# c. Visualizar la imagen con su predicción
def mostrar_prediccion(matriz, etiquetas, predicciones, indice):
    plt.figure(figsize=(2,2))
    plt.imshow(matriz[indice].reshape(28, 28), cmap='gray')
    plt.title(f"Real: {etiquetas[indice]}\nPred: {clases_predichas[indice]}")
    plt.axis('off')
    plt.show()

# Mostrar la primera imagen del conjunto de prueba con su predicción
mostrar_prediccion(X_test, y_test, clases_predichas, indice)


In [ ]:
# 15. Evaluación Más Detallada (Opcional)
# ------------------------------------

# Imprimir el reporte de clasificación
print("Reporte de Clasificación:\n")
print(classification_report(y_test, clases_predichas))

# Generar la matriz de confusión
cm = confusion_matrix(y_test, clases_predichas)

# Visualizar la matriz de confusión
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=np.unique(y_train), 
            yticklabels=np.unique(y_train))
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión con Data Augmentation')
plt.show()